In [1]:
import gym
from gym import spaces
import pygame
import numpy as np
import time
import matplotlib.pyplot as plt
import time
import random
import pandas as pd

#Set the seed for reproducibility
np.random.seed(7)
random.seed(7)

In [2]:
train = pd.read_excel("train.xlsx")
train.head()

,PRICES,Hour 01,Hour 02,Hour 03,Hour 04,Hour 05,Hour 06,Hour 07,Hour 08,Hour 09,...,Hour 15,Hour 16,Hour 17,Hour 18,Hour 19,Hour 20,Hour 21,Hour 22,Hour 23,Hour 24
0,2007-01-01,24.31,24.31,21.71,8.42,0.01,0.01,0.02,0.01,0.01,...,15.00,10.00,8.17,27.77,37.99,33.11,37.99,33.00,36.48,30.65
1,2007-01-02,16.01,11.00,9.01,7.50,9.00,7.45,16.50,28.01,29.96,...,37.57,36.45,37.99,53.00,59.69,50.09,50.00,36.22,31.09,29.84
2,2007-01-03,28.00,24.50,24.15,18.99,15.50,23.11,29.01,39.73,43.81,...,43.21,43.99,48.00,57.00,60.99,55.51,51.77,34.51,39.31,38.05
3,2007-01-04,31.01,28.69,27.00,24.20,12.02,27.00,28.00,34.85,41.31,...,40.01,38.00,42.66,53.00,58.99,53.50,44.01,35.00,34.46,31.57
4,2007-01-05,22.00,19.50,17.55,15.91,11.40,22.65,27.30,34.00,40.01,...,40.06,38.00,38.61,49.84,53.00,45.91,36.74,29.61,30.61,27.51


In [94]:
class SmartGridEnv(gym.Env):
    def __init__(self, price_data, battery_capacity=50, max_power=25, efficiency=0.9):
        super(SmartGridEnv, self).__init__()

        self.price_data = price_data
        self.battery_capacity = battery_capacity
        self.max_power = max_power
        self.efficiency = efficiency

        self.current_step = 0
        self.current_hour = 0
        self.current_battery = 50  # Initial battery level
        self.current_state = [self.current_step, self.current_hour, self.current_battery]
        self.done = False
        self.profit = 0  # Initial profit

        # Action space: 0 for buying, 1 for selling, 2 for holding
        self.action_space = gym.spaces.MultiDiscrete([3])

        # Observation space: current step, current hour, and current battery level
        self.observation_space = gym.spaces.MultiDiscrete([len(price_data), 24, battery_capacity])

    def reset(self):
        self.current_step = 0
        self.current_hour = 0
        self.current_battery = 50
        self.current_state = [self.current_step, self.current_hour, self.current_battery]
        self.done = False
        self.profit = 0  # Reset profit to 0
        return np.array(self.current_state)

    def step(self, action):
        if self.done:
            raise ValueError("Episode is done. Call reset() to start a new episode.")
    
        # Execute the action
        if action == 0:  # Charge
            if self.current_battery < self.battery_capacity:
                charge_cost = self.price_data.iloc[self.current_step, self.current_hour] * (self.max_power / self.efficiency)
                actual_charge = min(self.max_power, (self.battery_capacity - self.current_battery) * self.efficiency)
                self.current_battery += actual_charge / self.efficiency
                self.profit -= charge_cost
        elif action == 1 and self.current_battery > 0:  # Discharge only if there is energy in the battery
            discharge_amount = min(self.max_power / self.efficiency, self.current_battery)
            self.current_battery = max(self.current_battery - discharge_amount, 0)
            discharge_profit = discharge_amount * self.price_data.iloc[self.current_step, self.current_hour]
            self.profit += discharge_profit
        # No action needed for action == 2 (Hold)
    
        # Update time step and hour
        self.current_step += 1
        self.current_hour = self.current_step % 24
    
        # Update state
        self.current_state = [self.current_step, self.current_hour, self.current_battery]
    
        # Calculate reward (profit)
        reward = self.profit
    
        # Check if the episode is done (after 3 years of past electricity prices)
        if self.current_step >= len(self.price_data) - 1:
            self.done = True
    
        return self.current_state, reward, self.done, {}

    def render(self, mode="human"):
        if mode == "human":
            if not self.done:
                print(f"Step: {self.current_step}, Hour: {self.current_hour}, Battery: {self.current_battery}, Profit: {self.profit}")
            else:
                print("Episode finished.")
        elif mode == "pygame":
            self._render_pygame()
        elif mode == "rgb_array":
            raise NotImplementedError("Rendering as rgb_array is not supported for this environment.")

    def close(self):
        pass

In [95]:
# Extracting price columns
price_data = train.iloc[:, 1:]

# Create an instance of the environment
env = SmartGridEnv(price_data)

# Run the environment for a few steps
for _ in range(24):
    action = env.action_space.sample()  # Replace with your RL agent's action
    state, profit, done, _ = env.step(action)
    env.render(mode="human")  # Print the current state (for demonstration)
    if done:
        print("Episode finished.")
        break

# Reset the environment for a new episode
env.reset()

# Optionally, you can visualize the state after resetting
env.render(mode="human")

# Close the environment
env.close()
print()

Step: 1, Hour: 1, Battery: 50, Profit: 0
Step: 2, Hour: 2, Battery: 50, Profit: 0
Step: 3, Hour: 3, Battery: 50, Profit: 0
Step: 4, Hour: 4, Battery: 50, Profit: 0
Step: 5, Hour: 5, Battery: 50, Profit: 0
Step: 6, Hour: 6, Battery: 50, Profit: 0
Step: 7, Hour: 7, Battery: 22.22222222222222, Profit: 250.27777777777777
Step: 8, Hour: 8, Battery: 0.0, Profit: 1225.8333333333333
Step: 9, Hour: 9, Battery: 27.77777777777778, Profit: 114.72222222222217
Step: 10, Hour: 10, Battery: 50.0, Profit: -918.8888888888889
Step: 11, Hour: 11, Battery: 50.0, Profit: -918.8888888888889
Step: 12, Hour: 12, Battery: 50.0, Profit: -918.8888888888889
Step: 13, Hour: 13, Battery: 50.0, Profit: -918.8888888888889
Step: 14, Hour: 14, Battery: 22.22222222222222, Profit: -196.66666666666663
Step: 15, Hour: 15, Battery: 50.0, Profit: -1224.4444444444443
Step: 16, Hour: 16, Battery: 50.0, Profit: -1224.4444444444443
Step: 17, Hour: 17, Battery: 50.0, Profit: -1224.4444444444443
Step: 18, Hour: 18, Battery: 50.0, P

Note 1: not sure if (self.max_power / self.efficiency) is the right way to factor in efficiency as this increases MW. \
Note 2: Electricity is sold at the spot market price 
but is bought at twice the price to pay for the transmission costs and various taxes so when buying cost should be 2x. \
Note 3: reward maybe shouldn't be self.profit. If agent holds and proft is up it will be counted as a reward.\
Note 4: in step function remove np.array() from reward does something else with multidiscrete\
Note 5: discretize in agent instead of step function.

In [106]:
class QAgent():
    def __init__(self, env, discount_rate = 0.95, learning_rate=0.01, epsilon=0.05):
        self.env = env
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.action_space = 3
        self.Qtable = np.zeros([len(env.price_data), 24, env.battery_capacity])
        self.rewards = []

    def train(self, simulations = 1000, epsilon = 0.05):
        for i in range(simulations):
            while not self.env.done:
                current_state = self.env.current_state
                
                # pick random action
                if np.random.uniform(0,1) > 1-self.epsilon:
                    action = self.env.action_space.sample().item() # discretize
                # pick greedy action
                else:
                    day, hour = current_state[0],current_state[1]
                    action = np.argmax(self.Qtable[day,hour,:])

                next_state, reward, done, _ = env.step(action)
                # self.rewards.append(reward)
                
                # update Q_values
                Q_target = (reward + self.discount_rate*np.max(self.Qtable[next_state[0], next_state[1], :]))
                delta = self.learning_rate * (Q_target - self.Qtable[current_state[0], current_state[1], action])
                self.Qtable[current_state[0], current_state[1], action] = self.Qtable[current_state[0], current_state[1], action] + delta

            self.env.reset()

    def visulaize_rewards(self):
        # TODO
        pass

    def play_game(self):
        #TODO
        pass

env = SmartGridEnv(price_data)
agent = QAgent(env)
agent.train(simulations=100)
agent.Qtable[0, 0, :]

array([  6.05780166, 500.3306933 ,   1.25299486,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])